Okay I still should check that boltzmann stuff is working properly before doing the Q-values, so try running the boltzmann on the reward matrix rather than the q values and check that it gives you the distribution you'd expect just to make sure I'm not messing anything up there. 

Then if that's all fine, which it probably is, have another look at the Q-value and V-Value calculations in learn, and if they look right then step through and check that it's doing what I expect them to be doing. 

If it's something wrong with one of those then that might fix the learning side of things as well, but at least we'll have a proper trajectory to learn from because atm the one it's creating, and the resulting policy is abysmal, so fix that first, get the expert to perform more expertly and then you can check to see if there are problems with the learning algorithm :) 

Issue has to be in policy iteration somewhere 

In [ ]:
%reload_ext autoreload
from env import SingleStateSpace 
from generate_trajectory import generate_trajectory
from policy import Boltzmann 
from learn import compute_q_with_values
from RewardFunctions import SingleStateReward
import numpy as np 
from learn import policy_iteration
from utils import normalise_pi
from policy_walk import policy_walk 

In [ ]:

trajectory_length = 20 
#set up the agent and their policy 
env = SingleStateSpace(n_actions = 10, discount_rate = 0.1, R_max = 5)
#so we have access to env.reward
observation_times = np.cumsum(np.random.uniform(0, 2, size=trajectory_length))
print("Generated observation times:") 
print(observation_times)

#this isn't right but for now I'm just normalising pi - really it should be a boltzmann - later on I turn the Q-vals into a distribution via boltzmann 
#so it won't be able to learn i don't think 


# Use meshgrid to create a grid of all possible combinations of s, a, and t
s_grid, a_grid, t_grid = np.meshgrid(env.states, env.actions, observation_times, indexing='ij')
R = np.vectorize(env.reward)(s_grid,a_grid,t_grid)

# Calculate the reward for each combination using vectorized operations - oh I'm not actually using a boltzmann policy here I'm just using a normalised reward vector? 
(optimal_pi, optimal_values) = policy_iteration(env, len(observation_times), R)
optimal_q_values = np.ones((env.n_states, env.n_actions, len(observation_times)))
for s in range(env.n_states):
    for a in range(env.n_actions): 
        for t in range(len(observation_times)):
            optimal_q_values[s,a,t] = compute_q_with_values(env,s,a,t,optimal_values,R) 

boltzmann_policy = Boltzmann(optimal_q_values, env.actions)

#What is my initial policy? Everywhere I've seen, boltzmann is defined using the Q-values but you
#need a policy to find the Q-values so what is the actual policy? 
#Is it just like an exponential style distribution over the rewards? 
observations = generate_trajectory(env, boltzmann_policy, observation_times)

print("Generated trajectory") 
print(observations)

print("Running policy walk") 
learned_pi = policy_walk(env, observations, optimal_q_values) # there's a better way than passing in the Q* values but I cba dealing with it now 
print("finished learning policy")

In [ ]:
pi = np.ones((env.n_states,len(observations)))
import matplotlib.pyplot as plt 
for i in range(5):
    for s in range(env.n_states):
        for t in range(len(observations)): 
            pi[s,t] = boltzmann_policy(s,t)
    plt.plot(pi[0])
    plt.show()


In [ ]:
print(observation_times) 
for t in observation_times: # I should check I'm actually using the observation_times in the reward I bet I'm not and that'd have me shifted off by 2 each time which could cause the weird cycling rewards that I'm seeing 
    #nah it's not that, I create R properly and if I was using env.reward anywhere other than when I create R I'd be getting errors. 
    stuff = [] 
    for a_ in range(env.n_actions): 
        stuff.append(env.reward(0,a_,t))
    plt.plot(stuff)
    plt.show()
